In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Change directory to your code
%cd /content/drive/MyDrive/addition
%pwd   # verify you’re in the right place
!ls    # should show train.py, 4_operands_addition.txt, etc.

Mounted at /content/drive
/content/drive/MyDrive/addition
configuration_files	       legacy_code	   result_analysis.ipynb
configurator.py		       llama_adapter.py    result_analysis.py
data			       llama_tokenizer.py  result_analysis_script
data_generate.py	       main_utilities.py   results
data_generation_script	       model.py		   startHere2.ipynb
error_examples		       model_rope.py	   startHere3.ipynb
evaluation.py		       model_t5bias.py	   startHere.ipynb
extra_result_analysis_scripts  __pycache__	   statistical_measurements.py
gsm_test		       README.md	   train.py


# I. Generate Data (choose one synthetic task)

## Addition

In [ ]:
!python data_generate.py --task addition --num_operands 4 --experiment_name 4_operands_0_to_999_uniform --train_size 1000000 --test_size 10000 --val_size 10000 --train_eval True --sample-size 10000 --generate_reverse True

#### Ablation in Addition (e.g. randomize thousands-place of the output)

In [ ]:
!python data_generate.py --task addition --randomize thousands --num_operands 4 --experiment_name 4_operands_0_to_999_output_randomize_thousands --train_size 1000000 --test_size 10000 --val_size 10000 --train_eval True --sample-size 10000 --generate_reverse True


#### Addition with scratchpad (form 1)

In [ ]:
!python data_generate.py --task addition --reasoning_mode 1 --num_operands 4 --experiment_name 4_operands_0_to_999_uniform_scratchpad1 --train_size 1000000 --test_size 10000 --val_size 10000 --train_eval True --sample-size 10000 --generate_reverse True

#### Addition with scratchpad (form 2)

In [ ]:
!python data_generate.py --task addition --reasoning_mode 2 --num_operands 4 --experiment_name 4_operands_0_to_999_uniform_scratchpad2 --train_size 1000000 --test_size 10000 --val_size 10000 --train_eval True --sample-size 10000 --generate_reverse True

## Multiplication

In [ ]:
!python data_generate.py --task multiplication --experiment_name 40_digit_times_1_digit --train_size 1000000 --test_size 10000 --val_size 10000 \
--a_max_digits 40 --b_max_digits 1 --train_eval True --sample-size 10000 --generate_reverse True

## Comparison (Balanced data)

In [ ]:
!python data_generate.py --task comparison --experiment_name comparison_bal --train_eval True --sample-size 5000

## Sorting (Doubly balanced data)


In [ ]:
!python data_generate.py --task sorting --experiment_name 4_operands_sorting_doubly_balanced --train_eval True --sample-size 5000

# II. Let's Start Training!

#### The .txt file is the configuration file

## 4 Operands Addition

#### Reverse Output format

In [ ]:
!python train.py 4_operands_addition_reversed.txt

#### Plain output format

In [ ]:
!python train.py 4_operands_addition_plain.txt

#### Scratchpad Form 1

In [ ]:
!python train.py 4_operands_addition_plain_scratchpad1.txt

#### Scratchpad Form 2

In [ ]:
!python train.py 4_operands_addition_plain_scratchpad2.txt

#### Pythia Finetuning

In [ ]:
!python train.py 4_operands_addition_plain_pythia.txt

Using batch preparation method: per_example
Loading config file: /content/drive/MyDrive/addition/configuration_files/4_operands_addition_plain_pythia.txt
# Pythia-1B Configuration for 4-Operand Addition Task
# This config fine-tunes EleutherAI's Pythia-1B model on arithmetic tasks

# ===== Model Selection ===== #
use_llama = True  # Set to True to use pre-trained model
llama_model_name = "EleutherAI/pythia-1b"

# ===== Evaluation Settings ===== #
eval_interval = 2000  # More frequent eval for smaller model
eval_iters = 1
always_save_checkpoint = True

more_early_eval1 = True  # Enable frequent early evaluation
early_eval_interval1 = 25  # Evaluate every 25 iterations early on
early_eval_border1 = 100  # Until iteration 1000

# ===== WandB Logging ===== #
wandb_log = True
wandb_project = 'addition_pythia'
wandb_run_name = '4_operands_addition_plain_pythia_1b'

# ===== Model Architecture (Pythia-specific) ===== #
block_size = 64  # Larger for BPE efficiency
dropout = 0.1  # Light dropout

##

## Simpel Multiplication

In [ ]:
!python train.py 40_1_digits_mul_reversed.txt

## Comparison

In [ ]:
!python train.py comparison_bal.txt

## Sorting

In [ ]:
!python train.py 4_operands_sorting_doubly_bal.txt

## Slicing -- Addition

In [ ]:
!python train.py slicing_addition_4_operand_plain.txt --batch slicing

In [ ]:
!python train.py slicing_addition_4_operand_reverse.txt --batch slicing

## Positional Encoding

In [ ]:
!python train.py 4_operands_addition_reversed.txt --PE RoPE

In [ ]:
!python train.py 4_operands_addition_reversed.txt --PE t5

### Greedy Decoding

In [ ]:
!python train.py 4_operands_addition_reversed.txt --greedy

In [ ]:
!python train.py 4_operands_addition_plain.txt --greedy

# III. Result Analysis

## Addition Task

#### Digitwise Error Rates (4 operands addition)

In [ ]:
!python result_analysis_script/digitwise_error.py results/4_operands_0_to_999_uniform/reverse_out_test_garbage/4_operands_0_to_999_uniform_reverse/test_reverse_results.csv

/content/drive/MyDrive/addition/result_analysis_script/digitwise_error.py:109: DtypeWarning: Columns (3,4,5,9,15,19,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)
[(0, 'pred_iter_0'), (2000, 'pred_iter_2000'), (4000, 'pred_iter_4000'), (6000, 'pred_iter_6000'), (8000, 'pred_iter_8000'), (10000, 'pred_iter_10000'), (12000, 'pred_iter_12000'), (14000, 'pred_iter_14000'), (16000, 'pred_iter_16000'), (18000, 'pred_iter_18000'), (20000, 'pred_iter_20000'), (22000, 'pred_iter_22000'), (24000, 'pred_iter_24000'), (26000, 'pred_iter_26000'), (28000, 'pred_iter_28000'), (30000, 'pred_iter_30000'), (32000, 'pred_iter_32000'), (34000, 'pred_iter_34000'), (36000, 'pred_iter_36000'), (38000, 'pred_iter_38000'), (40000, 'pred_iter_40000'), (42000, 'pred_iter_42000'), (44000, 'pred_iter_44000'), (46000, 'pred_iter_46000'), (48000, 'pred_iter_48000'), (50000, 'pred_iter_50000'), (52000, 'pred_iter_52000'), (54000, 'pred_iter_54000'), (5600

#### Fit Normal

In [ ]:
!python result_analysis_script/fit_normal.py \
  --input results/4_operands_0_to_999_uniform/reverse_out_early_dense_eval/early_dense_eval_for_normal_distr_4_operands_0_to_999_uniform_reverse/test_reverse_results.csv \
  --iter-start 1000 --iter-end 1800 --iter-step 200 \
  --diff-min -800 --diff-max 800


In [ ]:
!python result_analysis_script/fit_normal.py \
  --input results/4_operands_0_to_999_uniform/reverse_out/4_operands_0_to_999_uniform_reverse/test_reverse_results.csv \
  --iter-start 8000 --iter-end 12000 --iter-step 2000 \
  --diff-min -100 --diff-max 100


In [ ]:
!python result_analysis_script/fit_normal.py \
  --input results/4_operands_0_to_999_uniform/reverse_out/4_operands_0_to_999_uniform_reverse/test_reverse_results.csv \
  --iter-start 60000 --iter-end 64000 --iter-step 2000 \
  --diff-min -20 --diff-max 20


#### Mutual Information Plot

In [ ]:
!python result_analysis_script/plot_mi_metrics.py \
  results/4_operands_0_to_999_uniform/reverse_out_complete_MI_1M_lines/4_operands_0_to_999_uniform_reverse/mi_metrics.csv

## Simple Multiplication Task

#### Digitwise Error (Simple multiplication, Colormap)

In [ ]:
!python result_analysis_script/mul_digitwise_error_colormap.py results/40_digit_times_1_digit/reverse_out/40_digit_times_1_digit/test_reverse_results.csv --max_steps 3000

## Comparison Task




#### Comparison Error Rate (Contrast Pairs)

In [ ]:
!python result_analysis_script/comparison_error_rate.py \
  results/comparison_bal/comparison_bal_1/thousands_diff_only_results.csv \
  results/comparison_bal/comparison_bal_1/hundreds_diff_only_results.csv \
  results/comparison_bal/comparison_bal_1/tens_diff_only_results.csv \
  results/comparison_bal/comparison_bal_1/units_diff_only_results.csv \
  --output_file_name contrast_pair_error_rate

Saved plot to results/comparison_bal/comparison_bal_1/contrast_pair_error_rate


## Sorting Task

#### Sorting Subskill from 10% to 90% Range

In [ ]:
!python result_analysis_script/sorting_acc_10_90_range.py \
  --csv \
    results/4_operands_sorting_doubly_balanced/conflicting_same_control_exp_correction/4_operands_sorting_doubly_balanced_conflicting_same_correction/test_results.csv \
    results/4_operands_sorting_doubly_balanced/conflicting_same_control_exp_correction/4_operands_sorting_doubly_balanced_conflicting_same_correction/digitwise_random_results.csv \
    results/4_operands_sorting_doubly_balanced/conflicting_same_control_exp_correction/4_operands_sorting_doubly_balanced_conflicting_same_correction/digitwise_thousand_results.csv \
    results/4_operands_sorting_doubly_balanced/conflicting_same_control_exp_correction/4_operands_sorting_doubly_balanced_conflicting_same_correction/digitwise_hundred_results.csv \
    results/4_operands_sorting_doubly_balanced/conflicting_same_control_exp_correction/4_operands_sorting_doubly_balanced_conflicting_same_correction/digitwise_ten_results.csv \
  --positions 1,2,3,4 \
  --mode length first second third fourth


#### Sorting Mixing Error

In [ ]:
!python result_analysis_script/mixing_error.py results/4_operands_sorting_doubly_balanced/conflicting_same_control_exp_correction_v2/4_operands_sorting_doubly_balanced_conflicting_same_correction_v2/1_3_same_2_4_agreeing_v2_results.csv


# IV. NanoGPT Scaling

#### 20M Parameters

In [ ]:
!python train.py 20M_4_operands_addition_reversed.txt

#### 100M Parameters

In [ ]:
!python train.py 100M_4_operands_addition_reversed.txt